In [1]:
%%capture

import ROOT
import os
from dask.distributed import Client
from utils import *
from variables import *
from samples import *
ROOT.RDF.Experimental.Distributed.open_files_locally = False
#logger = ROOT.RDF.Experimental.Distributed.create_logger("DEBUG")

redirector = "xrootd-cms.infn.it"
results_dir = "resultsRR2017_af-test_fix"

maxNfilespersample = 99999999

MT = False

distributed = True
nmaxpartitions = 90
sched_port = 20088

In [2]:
if distributed != True and MT == True:
    ROOT.ROOT.EnableImplicitMT()

# Import utils functions

In [3]:
'''
def my_initialization_function():
    ROOT.gInterpreter.Declare('{}'.format(data))
'''

"\ndef my_initialization_function():\n    ROOT.gInterpreter.Declare('{}'.format(data))\n"

In [4]:
#from dask.distributed import Client
#client = Client()  # set up local cluster on your laptop
#client

In [5]:
'''
if distributed == True:
    RDataFrame = ROOT.RDF.Experimental.Distributed.Dask.RDataFrame
    client = Client(address="tcp://127.0.0.1:"+str(sched_port))
    text_file = open("postselection.h", "r")
    data = text_file.read()
    #ROOT.RDF.Experimental.Distributed.initialize(my_initialization_function)

else:
    RDataFrame = ROOT.RDataFrame
    ROOT.gInterpreter.Declare('#include "postselection.h"')
'''

'\nif distributed == True:\n    RDataFrame = ROOT.RDF.Experimental.Distributed.Dask.RDataFrame\n    client = Client(address="tcp://127.0.0.1:"+str(sched_port))\n    text_file = open("postselection.h", "r")\n    data = text_file.read()\n    #ROOT.RDF.Experimental.Distributed.initialize(my_initialization_function)\n\nelse:\n    RDataFrame = ROOT.RDataFrame\n    ROOT.gInterpreter.Declare(\'#include "postselection.h"\')\n'

In [6]:
'''
if distributed == True:
    ROOT.RDF.Experimental.Distributed.initialize(my_initialization_function)
'''

'\nif distributed == True:\n    ROOT.RDF.Experimental.Distributed.initialize(my_initialization_function)\n'

In [7]:
text_file = open("postselection.h", "r")
data = text_file.read()
def my_initialization_function():
    ROOT.gInterpreter.Declare('{}'.format(data))
if distributed == True:
    RDataFrame = ROOT.RDF.Experimental.Distributed.Dask.RDataFrame
    client = Client(address="tcp://127.0.0.1:"+str(sched_port))
    ROOT.RDF.Experimental.Distributed.initialize(my_initialization_function)

else:
    RDataFrame = ROOT.RDataFrame
    my_initialization_function()

input_line_55:752:1: warning: control may reach end of non-void function [-Wreturn-type]
}
^


# Define analysis steps

In [8]:
def filter_trigger(df):
    df_goodvertex = df.Filter("Flag_goodVertices && Flag_globalSuperTightHalo2016Filter && Flag_HBHENoiseFilter && Flag_HBHENoiseIsoFilter && Flag_EcalDeadCellTriggerPrimitiveFilter && Flag_BadPFMuonFilter && Flag_ecalBadCalibFilterV2", "Good vertex selection")
    df_trigger = df_goodvertex.Filter("HLT_IsoMu27 || HLT_Mu50 || HLT_Ele35_WPTight_Gsf || HLT_Ele32_WPTight_Gsf_L1DoubleEG || HLT_Photon200", "Leptonic trigger")
    return df_trigger

def select_jets(df):
    df_atleast2Jets = df.Filter("nJet>=2", "At least two jets")
    df_GoodJets = df_atleast2Jets.Define("GoodJets_idx", "GoodJets(Jet_jetId, Jet_eta, Jet_pt, Jet_puId)")
    df_atleast2GoodJets = df_GoodJets.Filter("atleast2GoodJets(GoodJets_idx)", "At least two good jets")
    df_VBSjets = df_atleast2GoodJets.Define("VBSJet_idx", "SelectVBSJets_invmass(Jet_pt, Jet_eta, Jet_phi, Jet_mass, GoodJets_idx)")
    df_2VBSjets = df_VBSjets.Filter("VBSJet_idx[0] != VBSJet_idx[1]", "2 VBS jets")
    df_jetsDefinitions = df_2VBSjets.Define("Leadingjet_pt", "GetLeading(Jet_pt, VBSJet_idx)")\
                                    .Define("Leadingjet_eta", "GetLeading(Jet_eta, VBSJet_idx)")\
                                    .Define("Leadingjet_phi", "GetLeading(Jet_phi, VBSJet_idx)")\
                                    .Define("Leadingjet_mass", "GetLeading(Jet_mass, VBSJet_idx)")\
                                    .Define("Leadingjet_DeepFlv_b", "GetLeading(Jet_btagDeepFlavB, VBSJet_idx)")\
                                    .Define("Leadingjet_DeepCSVv2_b", "GetLeading(Jet_btagDeepB, VBSJet_idx)")\
                                    .Define("Leadingjet_CSVv2_b", "GetLeading(Jet_btagCSVV2, VBSJet_idx)")\
                                    .Define("SubLeadingjet_pt", "GetSubLeading(Jet_pt, VBSJet_idx)")\
                                    .Define("SubLeadingjet_eta", "GetSubLeading(Jet_eta, VBSJet_idx)")\
                                    .Define("SubLeadingjet_phi", "GetSubLeading(Jet_phi, VBSJet_idx)")\
                                    .Define("SubLeadingjet_mass", "GetSubLeading(Jet_mass, VBSJet_idx)")\
                                    .Define("SubLeadingjet_DeepFlv_b", "GetSubLeading(Jet_btagDeepFlavB, VBSJet_idx)")\
                                    .Define("SubLeadingjet_DeepCSVv2_b", "GetSubLeading(Jet_btagDeepB, VBSJet_idx)")\
                                    .Define("SubLeadingjet_CSVv2_b", "GetSubLeading(Jet_btagCSVV2, VBSJet_idx)")
    return df_jetsDefinitions

def select_lepton(df):
    df_selectElectron = df.Define("Electron_idx", "SelectElectron(Electron_pt, Electron_eta, Electron_phi, Electron_jetRelIso, Electron_mvaFall17V2Iso_WPL, Electron_mvaFall17V2Iso_WP90, Jet_eta, Jet_phi, VBSJet_idx)")
    df_selectMuon = df_selectElectron.Define("Muon_idx", "SelectMuon(Muon_pt, Muon_eta, Muon_phi, Muon_tightId, Muon_looseId, Muon_pfRelIso04_all, Jet_eta, Jet_phi, VBSJet_idx)")
    df_atLeast1Lepton = df_selectMuon.Filter("Electron_idx[1] != -1 || Muon_idx[1] != -1", "At least 1 at-least-loose lepton")
    df_goodLeptonFamily = df_atLeast1Lepton.Define("GoodLeptonFamily", "DetermineGoodLepton(HLT_IsoMu27, HLT_Mu50, HLT_Ele35_WPTight_Gsf, HLT_Ele32_WPTight_Gsf_L1DoubleEG, HLT_Photon200, HLT_PFHT250, HLT_PFHT350, Electron_idx, Electron_pt, Electron_eta, Electron_mvaFall17V2Iso_WPL, Electron_jetRelIso, Muon_idx, Muon_pt, Muon_eta, Muon_pfRelIso04_all, Muon_looseId)")
    df_compatibleLeptons = df_goodLeptonFamily.Filter("GoodLeptonFamily != -1 ", "Filter on leptons")
    df_leptonDefinitions = df_compatibleLeptons.Define("Lepton_pt", "GetLepton(Electron_pt, Electron_idx, Muon_corrected_pt, Muon_idx, GoodLeptonFamily)")\
                                                .Define("Lepton_eta", "GetLepton(Electron_eta, Electron_idx, Muon_eta, Muon_idx, GoodLeptonFamily)")\
                                                .Define("Lepton_phi", "GetLepton(Electron_phi, Electron_idx, Muon_phi, Muon_idx, GoodLeptonFamily)")\
                                                .Define("Lepton_mass", "GetLepton(Electron_mass, Electron_idx, Muon_mass, Muon_idx, GoodLeptonFamily)")\
                                                .Define("Lepton_charge", "GetLepton(Electron_charge, Electron_idx, Muon_charge, Muon_idx, GoodLeptonFamily)")\
                                                .Define("Lepton_genPartFlav", "GetLepton(Electron_genPartFlav, Electron_idx, Muon_genPartFlav, Muon_idx, GoodLeptonFamily)")\
                                                .Define("Lepton_TightRegion", "GetLeptonTightFlag(Electron_idx, Muon_idx, GoodLeptonFamily)")\
                                                .Define("Lepton_LnTRegion", "1 - Lepton_TightRegion")\
                                                .Define("Lepton_Zeppenfeld", "Zeppenfeld(Lepton_eta, Leadingjet_eta, SubLeadingjet_eta)")\
                                                .Define("Lepton_pdgId", "GetLepton(Electron_pdgId, Electron_idx, Muon_pdgId, Muon_idx, GoodLeptonFamily)")\
                                                .Define("Lepton_SFFake_vsjet2", "SFFakeRatio_lep_calc_vsjet2(Lepton_pt, Lepton_eta, Lepton_pdgId)")\
                                                .Define("Lepton_SFFake_vsjet4", "SFFakeRatio_lep_calc_vsjet4(Lepton_pt, Lepton_eta, Lepton_pdgId)")\
    
    return df_leptonDefinitions

def select_tau(df):
    df_selectTau = df.Define("Tau_idx", "SelectAndVetoTaus(Tau_pt, Tau_eta, Tau_phi, Tau_idDeepTau2017v2p1VSjet, Tau_idDeepTau2017v2p1VSe, Tau_idDeepTau2017v2p1VSmu, Tau_idDecayModeNewDMs, GoodLeptonFamily, Electron_idx, Electron_eta, Electron_phi, Muon_idx, Muon_eta, Muon_phi, Jet_eta, Jet_phi, VBSJet_idx)")
    df_1tau = df_selectTau.Filter("Tau_idx[1] != -1", "Exactly 1 at least loose Tau")
    df_tauDefinitions = df_1tau.Define("SelectedTau_pt", "GetTau(Tau_pt, Tau_idx)")\
                               .Define("SelectedTau_eta", "GetTau(Tau_eta, Tau_idx)")\
                               .Define("SelectedTau_phi", "GetTau(Tau_phi, Tau_idx)")\
                               .Define("SelectedTau_mass", "GetTau(Tau_mass, Tau_idx)")\
                               .Define("SelectedTau_charge", "GetTau(Tau_charge, Tau_idx)")\
                               .Define("SelectedTau_relleadtkpt", "GetTau(Tau_leadTkPtOverTauPt, Tau_idx)")\
                               .Define("SelectedTau_decayMode", "GetTau(Tau_decayMode, Tau_idx)")\
                               .Define("SelectedTau_isolation", "GetTau(Tau_neutralIso, Tau_idx)")\
                               .Define("SelectedTau_DeepTau_WP", "pow(GetTau(Tau_idDeepTau2017v2p1VSjet, Tau_idx)*1000., 2) + GetTau(Tau_idDeepTau2017v2p1VSjet, Tau_idx)*1000. + GetTau(Tau_idDeepTau2017v2p1VSe, Tau_idx)*1000.")\
                               .Define("SelectedTau_DeepTauVsEle_raw", "GetLog2(GetTau(Tau_idDeepTau2017v2p1VSe, Tau_idx) + 1)")\
                               .Define("SelectedTau_DeepTauVsMu_raw", "GetLog2(GetTau(Tau_idDeepTau2017v2p1VSmu, Tau_idx) + 1)")\
                               .Define("SelectedTau_DeepTauVsJet_raw", "GetLog2(GetTau(Tau_idDeepTau2017v2p1VSjet, Tau_idx) + 1)")\
                               .Define("SelectedTau_Zeppenfeld", "Zeppenfeld(SelectedTau_eta, Leadingjet_eta, SubLeadingjet_eta)")\
                               .Define("SelectedTau_TightRegion", "Tau_idx[1]")\
                               .Define("SelectedTau_LnTRegion", "1 - Tau_idx[1]")\
                               .Define("SelectedTau_SFFake_vsjet2", "SFFakeRatio_tau_calc_vsjet2(SelectedTau_pt, SelectedTau_eta)")\
                               .Define("SelectedTau_SFFake_vsjet4", "SFFakeRatio_tau_calc_vsjet4(SelectedTau_pt, SelectedTau_eta)")\
                               .Define("SelectedTau_genPartFlav", "GetTau(Tau_genPartFlav, Tau_idx)")\
                               .Define("SelectedTau_jetIdx", "GetTau(Tau_jetIdx, Tau_idx)")

                                #.Define("SelectedTau_GenMatch", "GetTau(Tau_genPartFlav, Tau_idx)")\
    
    return df_tauDefinitions

def scale_factors(df):
    #df_SF = df.Define("wzero", "1")
    df_SF = df.Define("LepSF", "GetLeptonSF(Electron_effSF, Electron_idx, Muon_effSF, Muon_idx, GoodLeptonFamily, IsMC)")\
              .Define("TauSF", "getTauSF(SelectedTau_pt, SelectedTau_eta, SelectedTau_genPartFlav, IsMC)")\
              .Define("Tau_vsjet_Down", "TauSF[0][0]")\
              .Define("Tau_vsjet_SF", "TauSF[0][1]")\
              .Define("Tau_vsjet_Up", "TauSF[0][2]")\
              .Define("Tau_vsele_Down", "TauSF[1][0]")\
              .Define("Tau_vsele_SF", "TauSF[1][1]")\
              .Define("Tau_vsele_Up", "TauSF[1][2]")\
              .Define("Tau_vsmu_Down", "TauSF[2][0]")\
              .Define("Tau_vsmu_SF", "TauSF[2][1]")\
              .Define("Tau_vsmu_Up", "TauSF[2][2]")\
              .Define("btagSFs", "btagcalc(GoodJets_idx, Jet_pt, Jet_eta, Jet_partonFlavour, Jet_btagDeepFlavB, Jet_btagSF_deepjet_M_up, Jet_btagSF_deepjet_M_down, Jet_btagSF_deepjet_M, Jet_btagDeepB, IsMC)")\
              .Define("btagSF", "btagSFs[0]")\
              .Define("btagUp", "btagSFs[1]")\
              .Define("btagDown", "btagSFs[2]")\
              .Define("mistagUp", "btagSFs[3]")\
              .Define("mistagDown", "btagSFs[4]")\
              .Define("w_SF", "PrefireWeight*puWeight*LepSF*Tau_vsjet_SF*Tau_vsele_SF*Tau_vsmu_SF*btagSF")

    
    
    return df_SF

def corrections(df):
    df_corrections = df.Define("FESs", "getFES(SelectedTau_eta, SelectedTau_decayMode, SelectedTau_genPartFlav, IsMC)")\
                       .Define("FESDown", "FESs[0]")\
                       .Define("FESSF", "FESs[1]")\
                       .Define("FESUp", "FESs[2]")\
                       .Define("TESs", "getTES(SelectedTau_pt, SelectedTau_decayMode, SelectedTau_genPartFlav, IsMC)")\
                       .Define("TESDown", "TESs[0]")\
                       .Define("TESSF", "TESs[1]")\
                       .Define("TESUp", "TESs[2]")\
                       .Define("SelectedTau_pt_corrected", "SelectedTau_pt*FESSF*TESSF")\
                       .Define("SelectedTau_mass_corrected", "SelectedTau_mass*FESSF*TESSF")
    
    return df_corrections

def general_definitions(df):
    df_definitions = df.Define("mjj", "GetInvMass(Jet_pt, Jet_eta, Jet_phi, Jet_mass, VBSJet_idx)")\
                       .Define("deltaPhi_jj", "deltaPhi(Leadingjet_phi, SubLeadingjet_phi)")\
                       .Define("deltaPhi_taulep", "deltaPhi(SelectedTau_phi, Lepton_phi)")\
                       .Define("deltaPhi_tauj1", "deltaPhi(SelectedTau_phi, Leadingjet_phi)")\
                       .Define("deltaPhi_tauj2", "deltaPhi(Leadingjet_phi, SubLeadingjet_phi)")\
                       .Define("deltaPhi_lepj1", "deltaPhi(Lepton_phi, Leadingjet_phi)")\
                       .Define("deltaPhi_lepj2", "deltaPhi(Lepton_phi, SubLeadingjet_phi)")\
                       .Define("deltaEta_jj", "Leadingjet_eta - SubLeadingjet_eta")\
                       .Define("deltaEta_taulep", "SelectedTau_eta - Lepton_eta")\
                       .Define("deltaEta_tauj1", "SelectedTau_eta - Leadingjet_eta")\
                       .Define("deltaEta_tauj2", "SelectedTau_eta - SubLeadingjet_eta")\
                       .Define("deltaEta_lepj1", "Lepton_eta - Leadingjet_eta")\
                       .Define("deltaEta_lepj2", "Lepton_eta - SubLeadingjet_eta")\
                       .Define("deltaTheta_jj", "deltaTheta(Leadingjet_pt, Leadingjet_eta, Leadingjet_phi, Leadingjet_mass, SubLeadingjet_pt, SubLeadingjet_eta, SubLeadingjet_phi, SubLeadingjet_mass)")\
                       .Define("deltaTheta_taulep", "deltaTheta(SelectedTau_pt_corrected,SelectedTau_eta, SelectedTau_phi, SelectedTau_mass_corrected, Lepton_pt, Lepton_eta, Lepton_phi, Lepton_mass)")\
                       .Define("deltaTheta_tauj1", "deltaTheta(SelectedTau_pt_corrected, SelectedTau_eta,SelectedTau_phi, SelectedTau_mass_corrected, Leadingjet_pt, Leadingjet_eta, Leadingjet_phi, Leadingjet_mass)")\
                       .Define("deltaTheta_tauj2", "deltaTheta(SelectedTau_pt_corrected, SelectedTau_eta, SelectedTau_phi, SelectedTau_mass_corrected, SubLeadingjet_pt, SubLeadingjet_eta, SubLeadingjet_phi, SubLeadingjet_mass)")\
                       .Define("Event_Zeppenfeld", "0.5*(Lepton_Zeppenfeld + SelectedTau_Zeppenfeld)")\
                       .Define("Lepton_Zeppenfeld_over_deltaEta_jj", "Lepton_Zeppenfeld / deltaEta_jj")\
                       .Define("SelectedTau_Zeppenfeld_over_deltaEta_jj", "SelectedTau_Zeppenfeld / deltaEta_jj")\
                       .Define("Event_Zeppenfeld_over_deltaEta_jj", "Event_Zeppenfeld / deltaEta_jj")\
                       .Define("mTlepMET", "sqrt(2*Lepton_pt*PuppiMET_pt*(1-cos(Lepton_phi-PuppiMET_phi)))")\
                       .Define("m_1T", "M1T(Lepton_pt, Lepton_eta, Lepton_phi, Lepton_mass, SelectedTau_pt_corrected, SelectedTau_eta, SelectedTau_phi, SelectedTau_mass_corrected, PuppiMET_pt, PuppiMET_phi)")\
                       .Define("m_o1", "Mo1(Lepton_pt, Lepton_eta, Lepton_phi, Lepton_mass, SelectedTau_pt_corrected, SelectedTau_eta, SelectedTau_phi, SelectedTau_mass_corrected, PuppiMET_pt, PuppiMET_phi)")\
                       .Define("event_SFFake_vsjet2", "GetEventSFFake(Lepton_SFFake_vsjet2, SelectedTau_SFFake_vsjet2, Lepton_LnTRegion, SelectedTau_LnTRegion)")\
                       .Define("event_SFFake_vsjet4", "GetEventSFFake(Lepton_SFFake_vsjet4, SelectedTau_SFFake_vsjet4, Lepton_LnTRegion, SelectedTau_LnTRegion)")\
                       .Define("m_jjtaulep","GetInvMassNoIndex4(Leadingjet_pt, Leadingjet_eta, Leadingjet_phi, Leadingjet_mass, SubLeadingjet_pt, SubLeadingjet_eta, SubLeadingjet_phi, SubLeadingjet_mass, SelectedTau_pt_corrected, SelectedTau_eta, SelectedTau_phi, SelectedTau_mass_corrected, Lepton_pt, Lepton_eta, Lepton_phi, Lepton_mass)")\
                       .Define("m_taulep","GetInvMassNoIndex(SelectedTau_pt_corrected, SelectedTau_eta, SelectedTau_phi, SelectedTau_mass_corrected, Lepton_pt, Lepton_eta, Lepton_phi, Lepton_mass)")\
                       .Define("mT_lep_MET","sqrt(2*Lepton_pt*PuppiMET_pt*(1. - cos(Lepton_phi-PuppiMET_phi)))")\
                       .Define("taujet_RelPt", "taujet_RelPt(SelectedTau_jetIdx, SelectedTau_pt, Jet_pt)")\
                       .Define("taujet_deltaPhi", "taujet_deltaPhi(SelectedTau_jetIdx, SelectedTau_phi, Jet_phi)")\
                       .Define("nBJet", "CountBJets(Jet_pt, Jet_eta, Jet_btagDeepFlavB)")\
                       .Define("event_RT","(Lepton_pt * SelectedTau_pt_corrected) / (Leadingjet_pt * SubLeadingjet_pt)")\
                       .Define("m_jjtau","GetInvMassNoIndex3(Leadingjet_pt, Leadingjet_eta, Leadingjet_phi, Leadingjet_mass, SubLeadingjet_pt, SubLeadingjet_eta, SubLeadingjet_phi, SubLeadingjet_mass, SelectedTau_pt_corrected, SelectedTau_eta, SelectedTau_phi, SelectedTau_mass_corrected)")\
                       .Define("SMbdt", "SMinference(Leadingjet_DeepFlv_b, SubLeadingjet_DeepFlv_b, Event_Zeppenfeld_over_deltaEta_jj, taujet_RelPt, m_o1, event_RT, taujet_deltaPhi, nJet, mT_lep_MET, m_jjtau, m_1T, nBJet, SubLeadingjet_pt, m_jjtaulep, mjj, Leadingjet_pt)");
                       #.Define("SMbdt", "SMinference(mjj, m_jjtaulep, m_taulep, mT_lep_MET, Leadingjet_pt, SubLeadingjet_pt, SelectedTau_mass, MET_pt)");

    return df_definitions

def book_histos(df, region, h):
    h[region] = {} 
    for item in aggregated_samples.items():
        h[region][item[0]] = {}
    h[region]["Fakes"] = {}
    for item in aggregated_samples.items():
        filter_string = ""
        for s in item[1]:
            #print(s.name)
            #print(sampleDict[s.name])
            if filter_string == "":
                filter_string += "Sample == {}".format(sampleDict[s.name])
            else:
                filter_string += " || Sample == {}".format(sampleDict[s.name])
        #print(filter_string)
        for v in variables[region]:
            df_sample = df.Filter(filter_string)
            if item[0] == "Data":
                h[region]["Data"][v._name] = {}
                h[region]["Fakes"][v._name] = {}
                h[region]['Data'][v._name]['etau'] = df_sample.Filter("GoodLeptonFamily==0")\
                                                   .Filter("event_SFFake_vsjet2 == 0.", "Both lepton and tau are prompt")\
                                                   .Filter("(abs(Leadingjet_eta)>3.2||abs(Leadingjet_eta)<2.5) && (abs(SubLeadingjet_eta)>3.2||abs(SubLeadingjet_eta)<2.5)", "Horn filter")\
                                                   .Histo1D(("h_" + v._name + "_" + region, v._title , v._nbins, v._bins), v._name)
                h[region]['Fakes'][v._name]['etau'] = df_sample.Filter("GoodLeptonFamily==0")\
                                                    .Filter("event_SFFake_vsjet2 > 0.", "At least one among lepton and tau is fake")\
                                                    .Filter("(abs(Leadingjet_eta)>3.2||abs(Leadingjet_eta)<2.5) && (abs(SubLeadingjet_eta)>3.2||abs(SubLeadingjet_eta)<2.5)", "Horn filter")\
                                                    .Histo1D(("h_" + v._name + "_" + region, v._title , v._nbins, v._bins), v._name, "event_SFFake_vsjet2")
                h[region]['Data'][v._name]['mutau'] = df_sample.Filter("GoodLeptonFamily==1")\
                                                   .Filter("event_SFFake_vsjet4 == 0.", "Both lepton and tau are prompt")\
                                                   .Filter("(abs(Leadingjet_eta)>3.2||abs(Leadingjet_eta)<2.5) && (abs(SubLeadingjet_eta)>3.2||abs(SubLeadingjet_eta)<2.5)", "Horn filter")\
                                                   .Histo1D(("h_" + v._name + "_" + region, v._title , v._nbins, v._bins), v._name)
                h[region]['Fakes'][v._name]['mutau'] = df_sample.Filter("GoodLeptonFamily==1")\
                                                    .Filter("event_SFFake_vsjet4 > 0.", "At least one among lepton and tau is fake")\
                                                    .Filter("(abs(Leadingjet_eta)>3.2||abs(Leadingjet_eta)<2.5) && (abs(SubLeadingjet_eta)>3.2||abs(SubLeadingjet_eta)<2.5)", "Horn filter")\
                                                    .Histo1D(("h_" + v._name + "_" + region, v._title , v._nbins, v._bins), v._name, "event_SFFake_vsjet4")
            else:
                h[region][item[0]][v._name] = {}
                h[region][item[0]][v._name]['etau'] = df_sample.Filter("GoodLeptonFamily==0").Filter("Lepton_TightRegion==1").Filter("(abs(Leadingjet_eta)>3.2||abs(Leadingjet_eta)<2.5) && (abs(SubLeadingjet_eta)>3.2||abs(SubLeadingjet_eta)<2.5)", "Horn filter").Filter("SelectedTau_TightRegion==1").Filter("(Lepton_genPartFlav==1|| Lepton_genPartFlav==15) && SelectedTau_genPartFlav==5", "prompt!").Histo1D(("h_" + v._name + "_" + region, v._title , v._nbins, v._bins), v._name, "wzero")
                h[region][item[0]][v._name]['mutau'] = df_sample.Filter("GoodLeptonFamily==1").Filter("Lepton_TightRegion==1").Filter("(abs(Leadingjet_eta)>3.2||abs(Leadingjet_eta)<2.5) && (abs(SubLeadingjet_eta)>3.2||abs(SubLeadingjet_eta)<2.5)", "Horn filter").Filter("SelectedTau_TightRegion==1").Filter("(Lepton_genPartFlav==1|| Lepton_genPartFlav==15) && SelectedTau_genPartFlav==5", "prompt!").Histo1D(("h_" + v._name + "_" + region, v._title , v._nbins, v._bins), v._name, "wzero")

# Define samples to be analyzed

In [9]:
sampleDict = {'WpWpJJ_EWK_2017': 0, 'WpWpJJ_QCD_2017': 1, 'VBS_SSWW_SM_2017': 2, 'VBS_SSWW_LL_SM_2017': 3, 'VBS_SSWW_TL_SM_2017': 4, 'VBS_SSWW_TT_SM_2017': 5, 'VBS_SSWW_cHW_2017': 6, 'VBS_SSWW_cHW_BSM_2017': 7, 'VBS_SSWW_cHW_INT_2017': 8, 'VBS_SSWW_cW_2017': 9, 'VBS_SSWW_cW_BSM_2017': 10, 'VBS_SSWW_cW_INT_2017': 11, 'VBS_SSWW_cW_cHW_2017': 12, 'VBS_SSWW_DIM6_2017': 13, 'VBS_SSWW_DIM6_SM_2017': 14, 'VBS_SSWW_cHW_SM_2017': 15, 'VBS_SSWW_cW_SM_2017': 16, 'VBS_SSWW_aQGC_2017': 17, 'VBS_SSWW_FS0_25_BSM_2017': 18, 'VBS_SSWW_FS0_25_INT_2017': 19, 'VBS_SSWW_FS0_25_SM_2017': 20, 'VBS_SSWW_FS0_25_2017': 21, 'VBS_SSWW_FS0_5_BSM_2017': 22, 'VBS_SSWW_FS0_5_INT_2017': 23, 'VBS_SSWW_FS0_5_SM_2017': 24, 'VBS_SSWW_FS0_5_2017': 25, 'VBS_SSWW_FS0_0_2017': 26, 'VBS_SSWW_FS1_50_BSM_2017': 27, 'VBS_SSWW_FS1_50_INT_2017': 28, 'VBS_SSWW_FS1_50_SM_2017': 29, 'VBS_SSWW_FS1_50_2017': 30, 'VBS_SSWW_FS1_10_BSM_2017': 31, 'VBS_SSWW_FS1_10_INT_2017': 32, 'VBS_SSWW_FS1_10_SM_2017': 33, 'VBS_SSWW_FS1_10_2017': 34, 'VBS_SSWW_FS1_0_2017': 35, 'VBS_SSWW_FM0_25_BSM_2017': 36, 'VBS_SSWW_FM0_25_INT_2017': 37, 'VBS_SSWW_FM0_25_SM_2017': 38, 'VBS_SSWW_FM0_25_2017': 39, 'VBS_SSWW_FM0_5_BSM_2017': 40, 'VBS_SSWW_FM0_5_INT_2017': 41, 'VBS_SSWW_FM0_5_SM_2017': 42, 'VBS_SSWW_FM0_5_2017': 43, 'VBS_SSWW_FM0_0_2017': 44, 'VBS_SSWW_FM1_25_BSM_2017': 45, 'VBS_SSWW_FM1_25_INT_2017': 46, 'VBS_SSWW_FM1_25_SM_2017': 47, 'VBS_SSWW_FM1_25_2017': 48, 'VBS_SSWW_FM1_5_BSM_2017': 49, 'VBS_SSWW_FM1_5_INT_2017': 50, 'VBS_SSWW_FM1_5_SM_2017': 51, 'VBS_SSWW_FM1_5_2017': 52, 'VBS_SSWW_FM1_0_2017': 53, 'VBS_SSWW_FM6_25_BSM_2017': 54, 'VBS_SSWW_FM6_25_INT_2017': 55, 'VBS_SSWW_FM6_25_SM_2017': 56, 'VBS_SSWW_FM6_25_2017': 57, 'VBS_SSWW_FM6_5_BSM_2017': 58, 'VBS_SSWW_FM6_5_INT_2017': 59, 'VBS_SSWW_FM6_5_SM_2017': 60, 'VBS_SSWW_FM6_5_2017': 61, 'VBS_SSWW_FM6_0_2017': 62, 'VBS_SSWW_FM7_50_BSM_2017': 63, 'VBS_SSWW_FM7_50_INT_2017': 64, 'VBS_SSWW_FM7_50_SM_2017': 65, 'VBS_SSWW_FM7_50_2017': 66, 'VBS_SSWW_FM7_10_BSM_2017': 67, 'VBS_SSWW_FM7_10_INT_2017': 68, 'VBS_SSWW_FM7_10_SM_2017': 69, 'VBS_SSWW_FM7_10_2017': 70, 'VBS_SSWW_FM7_0_2017': 71, 'VBS_SSWW_FT0_2p5_BSM_2017': 72, 'VBS_SSWW_FT0_2p5_INT_2017': 73, 'VBS_SSWW_FT0_2p5_SM_2017': 74, 'VBS_SSWW_FT0_2p5_2017': 75, 'VBS_SSWW_FT0_0p5_BSM_2017': 76, 'VBS_SSWW_FT0_0p5_INT_2017': 77, 'VBS_SSWW_FT0_0p5_SM_2017': 78, 'VBS_SSWW_FT0_0p5_2017': 79, 'VBS_SSWW_FT0_0_2017': 80, 'VBS_SSWW_FT1_1_BSM_2017': 81, 'VBS_SSWW_FT1_1_INT_2017': 82, 'VBS_SSWW_FT1_1_SM_2017': 83, 'VBS_SSWW_FT1_1_2017': 84, 'VBS_SSWW_FT1_0p2_BSM_2017': 85, 'VBS_SSWW_FT1_0p2_INT_2017': 86, 'VBS_SSWW_FT1_0p2_SM_2017': 87, 'VBS_SSWW_FT1_0p2_2017': 88, 'VBS_SSWW_FT1_0_2017': 89, 'VBS_SSWW_FT2_2p5_BSM_2017': 90, 'VBS_SSWW_FT2_2p5_INT_2017': 91, 'VBS_SSWW_FT2_2p5_SM_2017': 92, 'VBS_SSWW_FT2_2p5_2017': 93, 'VBS_SSWW_FT2_0p5_BSM_2017': 94, 'VBS_SSWW_FT2_0p5_INT_2017': 95, 'VBS_SSWW_FT2_0p5_SM_2017': 96, 'VBS_SSWW_FT2_0p5_2017': 97, 'VBS_SSWW_FT2_0_2017': 98, 'DHiggsToWW_2017': 99, 'sm_2017': 100, 'sm_lin_quad_cW_2017': 101, 'quad_cW_2017': 102, 'sm_lin_quad_cHW_2017': 103, 'quad_cHW_2017': 104, 'QCD_2017': 105, 'QCDHT_100to200_2017': 106, 'QCDHT_200to300_2017': 107, 'QCDHT_300to500_2017': 108, 'QCDHT_500to700_2017': 109, 'QCDHT_700to1000_2017': 110, 'QCDHT_1000to1500_2017': 111, 'QCDHT_1500to2000_2017': 112, 'QCDHT_2000toInf_2017': 113, 'ZZtoLep_2017': 114, 'ZZTo2L2Nu_2017': 115, 'ZZJJTo4L_EWK_2017': 116, 'ZZJJTo4L_QCD_2017': 117, 'GluGluToContinToZZTo2e2nu_2017': 118, 'GluGluToContinToZZTo2e2mu_2017': 119, 'GluGluToContinToZZTo2e2tau_2017': 120, 'GluGluToContinToZZTo2mu2nu_2017': 121, 'GluGluToContinToZZTo2mu2tau_2017': 122, 'GluGluToContinToZZTo4e_2017': 123, 'GluGluToContinToZZTo4mu_2017': 124, 'GluGluToContinToZZTo4tau_2017': 125, 'GluGluToContinToZZTo4L_2017': 126, 'TT_2017': 127, 'TT_SemiLep2017': 128, 'TT_Had_2017': 129, 'TTTo2L2Nu_2017': 130, 'TT_beff_2017': 131, 'WJets_2017': 132, 'WJetsHT70to100_2017': 133, 'WJetsHT100to200_2017': 134, 'WJetsHT200to400_2017': 135, 'WJetsHT400to600_2017': 136, 'WJetsHT600to800_2017': 137, 'WJetsHT800to1200_2017': 138, 'WJetsHT1200to2500_2017': 139, 'WJetsHT2500toInf_2017': 140, 'WJets_Fake_2017': 141, 'WJetsHT70to100_Fake_2017': 142, 'WJetsHT100to200_Fake_2017': 143, 'WJetsHT200to400_Fake_2017': 144, 'WJetsHT400to600_Fake_2017': 145, 'WJetsHT600to800_Fake_2017': 146, 'WJetsHT800to1200_Fake_2017': 147, 'WJetsHT1200to2500_Fake_2017': 148, 'WJetsHT2500toInf_Fake_2017': 149, 'DYJetsToLL_2017': 150, 'DY1JetsToLL_2017': 151, 'DY2JetsToLL_2017': 152, 'DY3JetsToLL_2017': 153, 'DY4JetsToLL_2017': 154, 'DYJetsToLLM5to50_2017': 155, 'DYJetsToLL_Fake_2017': 156, 'DY1JetsToLL_Fake_2017': 157, 'DY2JetsToLL_Fake_2017': 158, 'DY3JetsToLL_Fake_2017': 159, 'DY4JetsToLL_Fake_2017': 160, 'VG_2017': 161, 'ZG_2017': 162, 'WG_2017': 163, 'TVX_2017': 164, 'TTGJets_2017': 165, 'TTZToQQ_2017': 166, 'TTZToLLNuNu_2017': 167, 'TTWJetsToQQ_2017': 168, 'TTWJetsToLNu_2017': 169, 'tZq_ll_4f_2017': 170, 'WrongSign_2017': 171, 'WWto2L2Nu_2017': 172, 'GluGluToWWToENuENu_2017': 173, 'GluGluToWWToENuMNu2017': 174, 'GluGluToWWToENuTNu2017': 175, 'GluGluToWWToMNuENu_2017': 176, 'GluGluToWWToMNuMNu2017': 177, 'GluGluToWWToMNuTNu2017': 178, 'GluGluToWWToTNuENu_2017': 179, 'GluGluToWWToTNuMNu2017': 180, 'GluGluToWWToTNuTNu2017': 181, 'STtW_top_2017': 182, 'GluGluHToWWTo2L2Nu_2017': 183, 'GluGluHToZZTo2L2Q_2017': 184, 'GluGluHToZZTo4L_2017': 185, 'GluGluHToTauTau_2017': 186, 'VBFHToWWTo2L2Nu_2017': 187, 'VBFHToTauTau_2017': 188, 'ttHToNonbb_2017': 189, 'VHToNonbb_2017': 190, 'OtherWS_2017': 191, 'Other_2017': 192, 'WWTo2L2Nu_DoubleScattering_2017': 193, 'WWW_4F_2017': 194, 'WWZTo3L1Nu2Q_2017': 195, 'WZZ_2017': 196, 'ZZZ_2017': 197, 'WWG_2017': 198, 'WZ_2017': 199, 'FakeMu_2017': 200, 'DataMu_2017': 201, 'DataMuB_2017': 202, 'DataMuC_2017': 203, 'DataMuD_2017': 204, 'DataMuE_2017': 205, 'DataMuF_2017': 206, 'DataMuFake_2017': 207, 'FakeEle_2017': 208, 'FakeMuPromptTau_2017': 209, 'PromptMuFakeTau_2017': 210, 'FakeMuFakeTau_2017': 211, 'DataEle_2017': 212, 'DataEleB_2017': 213, 'DataEleC_2017': 214, 'DataEleD_2017': 215, 'DataEleE_2017': 216, 'DataEleF_2017': 217, 'DataEleFake_2017': 218, 'MCFake_2017': 219, 'FakeElePromptTau_2017': 220, 'PromptEleFakeTau_2017': 221, 'FakeEleFakeTau_2017': 222, 'DataEleMu_2017': 223, 'FakeEleMu_2017': 224, 'DataTau_2017': 225, 'DataTauB_2017': 226, 'DataTauC_2017': 227, 'DataTauD_2017': 228, 'DataTauE_2017': 229, 'DataTauF_2017': 230, 'DataTauFake_2017': 231, 'DataHT_2017': 232, 'DataHTnoB_2017': 233, 'DataHTB_2017': 234, 'DataHTC_2017': 235, 'DataHTD_2017': 236, 'DataHTE_2017': 237, 'DataHTF_2017': 238, 'DataMET_2017': 239, 'DataMETB_2017': 240, 'DataMETC_2017': 241, 'DataMETD_2017': 242, 'DataMETE_2017': 243, 'DataMETF_2017': 244, 'SampleEleFake_2017': 245, 'SampleMuFake_2017': 246, 'SampleHTFake_2017': 247, 'SampleHTFakepart_2017': 248, 'WpWpJJ_EWK_2018': 249, 'WpWpJJ_QCD_2018': 250, 'VBS_SSWW_SM_2018': 251, 'VBS_SSWW_LL_SM_2018': 252, 'VBS_SSWW_TL_SM_2018': 253, 'VBS_SSWW_TT_SM_2018': 254, 'VBS_SSWW_cHW_2018': 255, 'VBS_SSWW_cHW_BSM_2018': 256, 'VBS_SSWW_cHW_INT_2018': 257, 'VBS_SSWW_cW_2018': 258, 'VBS_SSWW_cW_BSM_2018': 259, 'VBS_SSWW_cW_INT_2018': 260, 'VBS_SSWW_cW_cHW_2018': 261, 'VBS_SSWW_DIM6_2018': 262, 'VBS_SSWW_DIM6_SM_2018': 263, 'VBS_SSWW_cHW_SM_2018': 264, 'VBS_SSWW_cW_SM_2018': 265, 'VBS_SSWW_aQGC_2018': 266, 'VBS_SSWW_FS0_25_BSM_2018': 267, 'VBS_SSWW_FS0_25_INT_2018': 268, 'VBS_SSWW_FS0_25_SM_2018': 269, 'VBS_SSWW_FS0_25_2018': 270, 'VBS_SSWW_FS0_5_BSM_2018': 271, 'VBS_SSWW_FS0_5_INT_2018': 272, 'VBS_SSWW_FS0_5_SM_2018': 273, 'VBS_SSWW_FS0_5_2018': 274, 'VBS_SSWW_FS0_0_2018': 275, 'VBS_SSWW_FS1_50_BSM_2018': 276, 'VBS_SSWW_FS1_50_INT_2018': 277, 'VBS_SSWW_FS1_50_SM_2018': 278, 'VBS_SSWW_FS1_50_2018': 279, 'VBS_SSWW_FS1_10_BSM_2018': 280, 'VBS_SSWW_FS1_10_INT_2018': 281, 'VBS_SSWW_FS1_10_SM_2018': 282, 'VBS_SSWW_FS1_10_2018': 283, 'VBS_SSWW_FS1_0_2018': 284, 'VBS_SSWW_FM0_25_BSM_2018': 285, 'VBS_SSWW_FM0_25_INT_2018': 286, 'VBS_SSWW_FM0_25_SM_2018': 287, 'VBS_SSWW_FM0_25_2018': 288, 'VBS_SSWW_FM0_5_BSM_2018': 289, 'VBS_SSWW_FM0_5_INT_2018': 290, 'VBS_SSWW_FM0_5_SM_2018': 291, 'VBS_SSWW_FM0_5_2018': 292, 'VBS_SSWW_FM0_0_2018': 293, 'VBS_SSWW_FM1_25_BSM_2018': 294, 'VBS_SSWW_FM1_25_INT_2018': 295, 'VBS_SSWW_FM1_25_SM_2018': 296, 'VBS_SSWW_FM1_25_2018': 297, 'VBS_SSWW_FM1_5_BSM_2018': 298, 'VBS_SSWW_FM1_5_INT_2018': 299, 'VBS_SSWW_FM1_5_SM_2018': 300, 'VBS_SSWW_FM1_5_2018': 301, 'VBS_SSWW_FM1_0_2018': 302, 'VBS_SSWW_FM6_25_BSM_2018': 303, 'VBS_SSWW_FM6_25_INT_2018': 304, 'VBS_SSWW_FM6_25_SM_2018': 305, 'VBS_SSWW_FM6_25_2018': 306, 'VBS_SSWW_FM6_5_BSM_2018': 307, 'VBS_SSWW_FM6_5_INT_2018': 308, 'VBS_SSWW_FM6_5_SM_2018': 309, 'VBS_SSWW_FM6_5_2018': 310, 'VBS_SSWW_FM6_0_2018': 311, 'VBS_SSWW_FM7_50_BSM_2018': 312, 'VBS_SSWW_FM7_50_INT_2018': 313, 'VBS_SSWW_FM7_50_SM_2018': 314, 'VBS_SSWW_FM7_50_2018': 315, 'VBS_SSWW_FM7_10_BSM_2018': 316, 'VBS_SSWW_FM7_10_INT_2018': 317, 'VBS_SSWW_FM7_10_SM_2018': 318, 'VBS_SSWW_FM7_10_2018': 319, 'VBS_SSWW_FM7_0_2018': 320, 'VBS_SSWW_FT0_2p5_BSM_2018': 321, 'VBS_SSWW_FT0_2p5_INT_2018': 322, 'VBS_SSWW_FT0_2p5_SM_2018': 323, 'VBS_SSWW_FT0_2p5_2018': 324, 'VBS_SSWW_FT0_0p5_BSM_2018': 325, 'VBS_SSWW_FT0_0p5_INT_2018': 326, 'VBS_SSWW_FT0_0p5_SM_2018': 327, 'VBS_SSWW_FT0_0p5_2018': 328, 'VBS_SSWW_FT0_0_2018': 329, 'VBS_SSWW_FT1_1_BSM_2018': 330, 'VBS_SSWW_FT1_1_INT_2018': 331, 'VBS_SSWW_FT1_1_SM_2018': 332, 'VBS_SSWW_FT1_1_2018': 333, 'VBS_SSWW_FT1_0p2_BSM_2018': 334, 'VBS_SSWW_FT1_0p2_INT_2018': 335, 'VBS_SSWW_FT1_0p2_SM_2018': 336, 'VBS_SSWW_FT1_0p2_2018': 337, 'VBS_SSWW_FT1_0_2018': 338, 'VBS_SSWW_FT2_2p5_BSM_2018': 339, 'VBS_SSWW_FT2_2p5_INT_2018': 340, 'VBS_SSWW_FT2_2p5_SM_2018': 341, 'VBS_SSWW_FT2_2p5_2018': 342, 'VBS_SSWW_FT2_0p5_BSM_2018': 343, 'VBS_SSWW_FT2_0p5_INT_2018': 344, 'VBS_SSWW_FT2_0p5_SM_2018': 345, 'VBS_SSWW_FT2_0p5_2018': 346, 'VBS_SSWW_FT2_0_2018': 347, 'DHiggsToWW_2018': 348, 'sm_2018': 349, 'sm_lin_quad_cW_2018': 350, 'quad_cW_2018': 351, 'sm_lin_quad_cHW_2018': 352, 'quad_cHW_2018': 353, 'QCD_2018': 354, 'QCDHT_100to200_2018': 355, 'QCDHT_200to300_2018': 356, 'QCDHT_300to500_2018': 357, 'QCDHT_500to700_2018': 358, 'QCDHT_700to1000_2018': 359, 'QCDHT_1000to1500_2018': 360, 'QCDHT_1500to2000_2018': 361, 'QCDHT_2000toInf_2018': 362, 'ZZtoLep_2018': 363, 'ZZTo2L2Nu_2018': 364, 'ZZJJTo4L_EWK_2018': 365, 'ZZJJTo4L_QCD_2018': 366, 'GluGluToContinToZZTo2e2nu_2018': 367, 'GluGluToContinToZZTo2e2mu_2018': 368, 'GluGluToContinToZZTo2e2tau_2018': 369, 'GluGluToContinToZZTo2mu2nu_2018': 370, 'GluGluToContinToZZTo2mu2tau_2018': 371, 'GluGluToContinToZZTo4e_2018': 372, 'GluGluToContinToZZTo4mu_2018': 373, 'GluGluToContinToZZTo4tau_2018': 374, 'GluGluToContinToZZTo4L_2018': 375, 'TT_2018': 376, 'TT_SemiLep_2018': 377, 'TT_Had_2018': 378, 'WJets_2018': 379, 'WJetsHT70to100_2018': 380, 'WJetsHT100to200_2018': 381, 'WJetsHT200to400_2018': 382, 'WJetsHT400to600_2018': 383, 'WJetsHT600to800_2018': 384, 'WJetsHT800to1200_2018': 385, 'WJetsHT1200to2500_2018': 386, 'WJetsHT2500toInf_2018': 387, 'DYJetsToLL_2018': 388, 'DY1JetsToLL_2018': 389, 'DY2JetsToLL_2018': 390, 'DY3JetsToLL_2018': 391, 'DY4JetsToLL_2018': 392, 'SampleHTFake_2018': 393, 'VG_2018': 394, 'ZG_2018': 395, 'WG_2018': 396, 'TVX_2018': 397, 'TTGJets_2018': 398, 'TTZToQQ_2018': 399, 'TTZToLLNuNu_2018': 400, 'TTWJetsToQQ_2018': 401, 'TTWJetsToLNu_2018': 402, 'tZq_ll_4f_2018': 403, 'WrongSign_2018': 404, 'WWto2L2Nu_2018': 405, 'GluGluToWWToENuENu_2018': 406, 'GluGluToWWToMNuENu_2018': 407, 'GluGluToWWToTNuENu_2018': 408, 'STtW_top_2018': 409, 'GluGluHToWWTo2L2Nu_2018': 410, 'GluGluHToZZTo2L2Q_2018': 411, 'GluGluHToZZTo4L_2018': 412, 'GluGluHToTauTau_2018': 413, 'VBFHToWWTo2L2Nu_2018': 414, 'VBFHToTauTau_2018': 415, 'ttHToNonbb_2018': 416, 'VHToNonbb_2018': 417, 'OtherWS_2018': 418, 'Other_2018': 419, 'WWTo2L2Nu_DoubleScattering_2018': 420, 'WWW_4F_2018': 421, 'WWZTo3L1Nu2Q_2018': 422, 'WZZ_2018': 423, 'ZZZ_2018': 424, 'WWG_2018': 425, 'TTTo2L2Nu_2018': 426, 'WZ_2018': 427, 'FakeMu_2018': 428, 'DataMu_2018': 429, 'DataMuA_2018': 430, 'DataMuB_2018': 431, 'DataMuC_2018': 432, 'DataMuD_2018': 433, 'FakeEle_2018': 434, 'FakeMuPromptTau_2018': 435, 'PromptMuFakeTau_2018': 436, 'FakeMuFakeTau_2018': 437, 'DataEle_2018': 438, 'DataEleA_2018': 439, 'DataEleB_2018': 440, 'DataEleC_2018': 441, 'DataEleD_2018': 442, 'FakeElePromptTau_2018': 443, 'PromptEleFakeTau_2018': 444, 'FakeEleFakeTau_2018': 445, 'DataHT_2018': 446, 'DataHTA_2018': 447, 'DataHTB_2018': 448, 'DataHTC_2018': 449, 'DataHTD_2018': 450}
'''
aggregated_samples = {
                      'Other': [WWTo2L2Nu_DoubleScattering_2017, WWW_4F_2017, WWZTo3L1Nu2Q_2017, WZZ_2017, ZZZ_2017, WWG_2017], 
                      'tVX': [TTGJets_2017, TTZToQQ_2017, TTZToLLNuNu_2017, TTWJetsToQQ_2017, TTWJetsToLNu_2017, tZq_ll_4f_2017], 
                      'QCD_ssWW': [WpWpJJ_QCD_2017],
                      'Vgamma':  [ZG_2017, WG_2017],
                      'ZZ': [ZZTo2L2Nu_2017, ZZJJTo4L_EWK_2017, ZZJJTo4L_QCD_2017, GluGluToContinToZZTo2e2nu_2017, 
                             GluGluToContinToZZTo2e2mu_2017, GluGluToContinToZZTo2e2tau_2017, 
                             GluGluToContinToZZTo2mu2nu_2017, GluGluToContinToZZTo2mu2tau_2017, GluGluToContinToZZTo4e_2017, 
                             GluGluToContinToZZTo4mu_2017, GluGluToContinToZZTo4tau_2017], 
                      'WZ': [WZ_2017], 
                      'VBS_ssWW': [VBS_SSWW_LL_SM_2017, VBS_SSWW_TL_SM_2017, VBS_SSWW_TT_SM_2017], 
                      #'VBS ssWW cW (BSM + INT)': [VBS_SSWW_cW_BSM_2017, VBS_SSWW_cW_INT_2017],
                      #'VBS ssWW cHW (BSM + INT)': [VBS_SSWW_cHW_BSM_2017, VBS_SSWW_cHW_INT_2017],
                      'DY_jets': [DY1JetsToLL_2017, DY2JetsToLL_2017, DY3JetsToLL_2017, DY4JetsToLL_2017, DYJetsToLLM5to50_2017],
                      'Wrong_Sign': [WWto2L2Nu_2017, GluGluToWWToENuENu_2017, GluGluToWWToENuMNu2017, 
                                     GluGluToWWToENuTNu2017, GluGluToWWToMNuENu_2017, GluGluToWWToMNuMNu2017, 
                                     GluGluToWWToMNuTNu2017, GluGluToWWToTNuENu_2017, GluGluToWWToTNuMNu2017, 
                                     GluGluToWWToTNuTNu2017, STtW_top_2017, 
                                        GluGluHToWWTo2L2Nu_2017, 
                                        #GluGluHToZZTo2L2Q_2017,
                                        GluGluHToZZTo4L_2017,
                                        GluGluHToTauTau_2017, 
                                        VBFHToWWTo2L2Nu_2017, VBFHToTauTau_2017, ttHToNonbb_2017, VHToNonbb_2017], 
                      'ttbar DiLep': [TTTo2L2Nu_2017],  
                      'Data': [DataEleB_2017, DataEleC_2017, DataEleD_2017, DataEleE_2017, DataEleF_2017, 
                               DataMuB_2017, DataMuC_2017, DataMuD_2017, DataMuE_2017, DataMuF_2017], 
                     }
'''
aggregated_samples = {
    'Vgamma':  [ZG_2017, WG_2017],
    'QCD_ssWW': [WpWpJJ_QCD_2017],   
    'tVX': [TTGJets_2017, TTZToQQ_2017, TTZToLLNuNu_2017, TTWJetsToQQ_2017, TTWJetsToLNu_2017, tZq_ll_4f_2017], 
    'Other': [WWTo2L2Nu_DoubleScattering_2017, WWW_4F_2017, WWZTo3L1Nu2Q_2017, WZZ_2017, ZZZ_2017, WWG_2017], 
    'ttbar DiLep': [TTTo2L2Nu_2017], 
    'WZ': [WZ_2017], 
    'DY_jets': [DY1JetsToLL_2017, DY2JetsToLL_2017, DY3JetsToLL_2017, DY4JetsToLL_2017, DYJetsToLLM5to50_2017],
    'Wrong_Sign': [WWto2L2Nu_2017, GluGluToWWToENuENu_2017, GluGluToWWToENuMNu2017, GluGluToWWToENuTNu2017, GluGluToWWToMNuENu_2017, GluGluToWWToMNuMNu2017, GluGluToWWToMNuTNu2017, GluGluToWWToTNuENu_2017, GluGluToWWToTNuMNu2017, GluGluToWWToTNuTNu2017, STtW_top_2017, GluGluHToWWTo2L2Nu_2017, GluGluHToZZTo4L_2017,GluGluHToTauTau_2017, VBFHToWWTo2L2Nu_2017, VBFHToTauTau_2017, ttHToNonbb_2017, VHToNonbb_2017], 
    'ZZ': [ZZTo2L2Nu_2017, ZZJJTo4L_EWK_2017, ZZJJTo4L_QCD_2017, GluGluToContinToZZTo2e2nu_2017, GluGluToContinToZZTo2e2mu_2017, GluGluToContinToZZTo2e2tau_2017, GluGluToContinToZZTo2mu2nu_2017, GluGluToContinToZZTo2mu2tau_2017, GluGluToContinToZZTo4e_2017, GluGluToContinToZZTo4mu_2017, GluGluToContinToZZTo4tau_2017], 
    'VBS_ssWW': [VBS_SSWW_LL_SM_2017, VBS_SSWW_TL_SM_2017, VBS_SSWW_TT_SM_2017], 
    'Data': [DataEleB_2017, DataEleC_2017, DataEleD_2017, DataEleE_2017, DataEleF_2017, DataMuB_2017, DataMuC_2017, DataMuD_2017, DataMuE_2017, DataMuF_2017], 
}


In [10]:
chain = []
proxy = redirector
max_lines = maxNfilespersample

for v in aggregated_samples.values():
    for s in v:
        with open("macros/files/{}.txt".format(s.name)) as file:
            i = 0
            for line in file:
                if i < max_lines: 
                    line = line.rstrip()
                    splitted_parts = line.split("/")
                    splitted_parts[2] = proxy
                    line = "/".join(splitted_parts)
                    chain.append(line)
                i += 1
#chain

In [11]:
'''
empty_files = []
with open("empty_files.txt") as file:
    for line in file:
        line = line.rstrip()
        splitted_parts = line.split("/")
        #print(splitted_parts)
        splitted_parts[2] = proxy
        line = "/".join(splitted_parts)
        empty_files.append(line)

for j in empty_files:
    if j in chain:
        chain.remove(j)
'''

'\nempty_files = []\nwith open("empty_files.txt") as file:\n    for line in file:\n        line = line.rstrip()\n        splitted_parts = line.split("/")\n        #print(splitted_parts)\n        splitted_parts[2] = proxy\n        line = "/".join(splitted_parts)\n        empty_files.append(line)\n\nfor j in empty_files:\n    if j in chain:\n        chain.remove(j)\n'

In [12]:
#for j in empty_files:
#    if j in chain:
#        print(j)

# Do analysis

In [13]:
#df = ROOT.RDataFrame("Events", 'https://ttedesch.web.cern.ch/ttedesch/testAF/VBS_SSWW_LL_SM_2017/tree_hadd_2.root')
#print(df.GetColumnNames())

In [14]:
'''
def book_histos(df, region, h):
    h[region] = {} 
    for item in aggregated_samples.items():
        h[region][item[0]] = {}
    h[region]["Fakes"] = {}
    for item in aggregated_samples.items():
        filter_string = ""
        for s in item[1]:
            #print(s.name)
            #print(sampleDict[s.name])
            if filter_string == "":
                filter_string += "Sample == {}".format(sampleDict[s.name])
            else:
                filter_string += " || Sample == {}".format(sampleDict[s.name])
        #print(filter_string)
        for v in variables[region]:
            df_sample = df.Filter(filter_string)
            if item[0] == "Data":
                h[region]["Data"][v._name] = {}
                h[region]["Fakes"][v._name] = {}
                h[region]['Data'][v._name]['etau'] = df_sample.Filter("GoodLeptonFamily==0")\
                                                   .Filter("event_SFFake_vsjet2 == 0.", "Both lepton and tau are prompt")\
                                                   .Filter("(abs(Leadingjet_eta)>3.2||abs(Leadingjet_eta)<2.5) && (abs(SubLeadingjet_eta)>3.2||abs(SubLeadingjet_eta)<2.5)", "Horn filter")\
                                                   .Histo1D(("h_" + v._name + "_" + region, v._title , v._nbins, v._bins), v._name)
                h[region]['Fakes'][v._name]['etau'] = df_sample.Filter("GoodLeptonFamily==0")\
                                                    .Filter("event_SFFake_vsjet2 > 0.", "At least one among lepton and tau is fake")\
                                                    .Filter("(abs(Leadingjet_eta)>3.2||abs(Leadingjet_eta)<2.5) && (abs(SubLeadingjet_eta)>3.2||abs(SubLeadingjet_eta)<2.5)", "Horn filter")\
                                                    .Histo1D(("h_" + v._name + "_" + region, v._title , v._nbins, v._bins), v._name, "event_SFFake_vsjet2")
                h[region]['Data'][v._name]['mutau'] = df_sample.Filter("GoodLeptonFamily==1")\
                                                   .Filter("event_SFFake_vsjet4 == 0.", "Both lepton and tau are prompt")\
                                                   .Filter("(abs(Leadingjet_eta)>3.2||abs(Leadingjet_eta)<2.5) && (abs(SubLeadingjet_eta)>3.2||abs(SubLeadingjet_eta)<2.5)", "Horn filter")\
                                                   .Histo1D(("h_" + v._name + "_" + region, v._title , v._nbins, v._bins), v._name)
                h[region]['Fakes'][v._name]['mutau'] = df_sample.Filter("GoodLeptonFamily==1")\
                                                    .Filter("event_SFFake_vsjet4 > 0.", "At least one among lepton and tau is fake")\
                                                    .Filter("(abs(Leadingjet_eta)>3.2||abs(Leadingjet_eta)<2.5) && (abs(SubLeadingjet_eta)>3.2||abs(SubLeadingjet_eta)<2.5)", "Horn filter")\
                                                    .Histo1D(("h_" + v._name + "_" + region, v._title , v._nbins, v._bins), v._name, "event_SFFake_vsjet4")
            else:
                h[region][item[0]][v._name] = {}
                h[region][item[0]][v._name]['etau'] = df_sample.Filter("GoodLeptonFamily==0").Filter("Lepton_TightRegion==1").Filter("(abs(Leadingjet_eta)>3.2||abs(Leadingjet_eta)<2.5) && (abs(SubLeadingjet_eta)>3.2||abs(SubLeadingjet_eta)<2.5)", "Horn filter").Filter("SelectedTau_TightRegion==1").Filter("(Lepton_genPartFlav==1|| Lepton_genPartFlav==15) && SelectedTau_genPartFlav==5", "prompt!").Histo1D(("h_" + v._name + "_" + region, v._title , v._nbins, v._bins), v._name, "wzero")
                h[region][item[0]][v._name]['mutau'] = df_sample.Filter("GoodLeptonFamily==1").Filter("Lepton_TightRegion==1").Filter("(abs(Leadingjet_eta)>3.2||abs(Leadingjet_eta)<2.5) && (abs(SubLeadingjet_eta)>3.2||abs(SubLeadingjet_eta)<2.5)", "Horn filter").Filter("SelectedTau_TightRegion==1").Filter("(Lepton_genPartFlav==1|| Lepton_genPartFlav==15) && SelectedTau_genPartFlav==5", "prompt!").Histo1D(("h_" + v._name + "_" + region, v._title , v._nbins, v._bins), v._name, "wzero")
'''

'\ndef book_histos(df, region, h):\n    h[region] = {} \n    for item in aggregated_samples.items():\n        h[region][item[0]] = {}\n    h[region]["Fakes"] = {}\n    for item in aggregated_samples.items():\n        filter_string = ""\n        for s in item[1]:\n            #print(s.name)\n            #print(sampleDict[s.name])\n            if filter_string == "":\n                filter_string += "Sample == {}".format(sampleDict[s.name])\n            else:\n                filter_string += " || Sample == {}".format(sampleDict[s.name])\n        #print(filter_string)\n        for v in variables[region]:\n            df_sample = df.Filter(filter_string)\n            if item[0] == "Data":\n                h[region]["Data"][v._name] = {}\n                h[region]["Fakes"][v._name] = {}\n                h[region][\'Data\'][v._name][\'etau\'] = df_sample.Filter("GoodLeptonFamily==0")                                                   .Filter("event_SFFake_vsjet2 == 0.", "Both lepton and tau

In [15]:
h = {}

if distributed == True:
    df = RDataFrame("Events", chain, npartitions=nmaxpartitions, daskclient=client)
else:
    #chain = ['https://ttedesch.web.cern.ch/ttedesch/testAF/VBS_SSWW_LL_SM_2017/tree_hadd_4.root']
    #chain = ['root://212.189.205.221:30443//root://stormgf2.pi.infn.it//store/user/ttedesch/VBS_PG_dummy/SingleMuon/DataMuB_2017/211126_085107/0000/tree_hadd_2.root',]
    chain = ['https://ttedesch.web.cern.ch/ttedesch/testAF/VBS_SSWW_LL_SM_2017/tree_hadd_2.root']
    df = RDataFrame("Events", chain).Range(1000000)    

df_type = df.Define("IsMC", "isMC(Sample)").Define("Year", "2017")    

#trigger and VBS jets selections  and e/mu + tau final state
df_trigger = filter_trigger(df_type)
df_jets = select_jets(df_trigger)
df_lepton = select_lepton(df_jets).Define("PassDataLeptonCheck", "DataLeptonCheck(Sample, GoodLeptonFamily, IsMC)").Filter("PassDataLeptonCheck == true", "DataLeptonCheck")
df_tau = select_tau(df_lepton)
df_corrections = corrections(df_tau)
df_definitions = general_definitions(df_corrections)
df_SF = scale_factors(df_definitions).Define("Nevents", "getNevents(Sample, IsMC)")\
                                   .Define("lumi", "getLumi(Year, IsMC)")\
                                   .Define("xsec", "getXSec(Sample, IsMC)")\
                                   .Define("w_nominal", "lumi*xsec* 1000./Nevents")\
                                   .Define("wzero", "w_SF*w_nominal")


df_METCut_false = df_SF.Filter("PuppiMET_pt <= 50", "MET cut false")
df_METCut_false_SS= df_METCut_false.Filter("SameCharge(GoodLeptonFamily, Electron_idx, Electron_charge, Muon_idx, Muon_charge, Tau_idx, Tau_charge)", "SameSign")
df_METCut_false_OS= df_METCut_false.Filter("SameCharge(GoodLeptonFamily, Electron_idx, Electron_charge, Muon_idx, Muon_charge, Tau_idx, Tau_charge) == false", "OppositeSign")

df_METCut_true = df_SF.Filter("PuppiMET_pt > 50", "MET cut true")
df_METCut_true_SS = df_METCut_true.Filter("SameCharge(GoodLeptonFamily, Electron_idx, Electron_charge, Muon_idx, Muon_charge, Tau_idx, Tau_charge)", "SameSign")
df_METCut_true_OS = df_METCut_true.Filter("SameCharge(GoodLeptonFamily, Electron_idx, Electron_charge, Muon_idx, Muon_charge, Tau_idx, Tau_charge) == false", "OppositeSign")

##### CR non prompt
#df_NONPROMPT = df_METCut_false_SS.Filter("mT_lep_MET > 50")
df_NONPROMPT = df_METCut_false_SS
book_histos(df_NONPROMPT, 'CR_non_prompt', h)
##### CR QCD
#df_QCD = df_METCut_false_SS.Filter("mT_lep_MET <= 50")
#book_histos(df_QCD, 'CR_QCD', h)
##### SR 
df_SR = df_METCut_true_SS.Filter("BVeto(Jet_pt, Jet_eta, Jet_btagDeepFlavB, GoodJets_idx) == true", "Bveto").Filter("mjj > 500", "mjj cut")
#df_SR = df_SF.Filter("PuppiMET_pt > 40", "MET cut special true").Filter("SameCharge(GoodLeptonFamily, Electron_idx, Electron_charge, Muon_idx, Muon_charge, Tau_idx, Tau_charge)", "SameSign").Filter("BVeto(Jet_pt, Jet_eta, Jet_btagDeepFlavB, GoodJets_idx) == true", "Bveto").Filter("mjj > 500", "mjj cut")
book_histos(df_SR, 'SR', h)
#### CR ttbar
df_TTBAR = df_METCut_true_OS.Filter("BVeto(Jet_pt, Jet_eta, Jet_btagDeepFlavB, GoodJets_idx) == false", "not Bveto")
book_histos(df_TTBAR, 'CR_ttbar', h)
#### CR wrong sign 
df_WRONGSIGN = df_METCut_true_OS.Filter("BVeto(Jet_pt, Jet_eta, Jet_btagDeepFlavB, GoodJets_idx) == true", "Bveto")
book_histos(df_WRONGSIGN, 'CR_wrong_sign', h)
#### CR Z + jets
#df_Zjets = df_METCut_true_OS.Filter("BVeto(Jet_pt, Jet_eta, Jet_btagDeepFlavB, GoodJets_idx) == true", "Bveto")
df_Zjets = df_METCut_false_OS.Filter("BVeto(Jet_pt, Jet_eta, Jet_btagDeepFlavB, GoodJets_idx) == true", "Bveto")
book_histos(df_Zjets, 'CR_Zjets', h)

In [16]:
#df_definitions.Display("SMbdt").Print()

# Save histos

In [17]:
if results_dir not in os.listdir():
    os.mkdir("./" + results_dir)

In [18]:
%%time
import math

if "plots" not in os.listdir("./" + results_dir):
    os.mkdir("./" + results_dir +"/" + "plots")
folder = "./"+ results_dir + "/plots"
if "electron" not in os.listdir(folder):
    os.mkdir(folder + "/" + "electron")
if "muon" not in os.listdir(folder):
    os.mkdir(folder + "/" + "muon")
for lepton, final_state in zip(['electron', 'muon'], ['etau', 'mutau']):
    for region, samples_dict in h.items():
        for sample, histos_dict in samples_dict.items(): 
            file = ROOT.TFile(folder + "/" + lepton + "/" + "{}.root".format(sample), 'UPDATE')
            for feature, histo in histos_dict.items():
                #histo[final_state].Write()
                h1 = histo[final_state].GetValue()
                nbins = h1.GetNbinsX()
                h1.SetBinContent(1, h1.GetBinContent(0) + h1.GetBinContent(1))
                h1.SetBinError(1, math.sqrt(pow(h1.GetBinError(0),2) + pow(h1.GetBinError(1),2)))
                h1.SetBinContent(nbins, h1.GetBinContent(nbins) + h1.GetBinContent(nbins+1))
                h1.SetBinError(nbins, math.sqrt(pow(h1.GetBinError(nbins),2) + pow(h1.GetBinError(nbins+1),2)))
                if sample == "Fakes":
                    for bidx in range(nbins):          
                        bidx_l = bidx + 1
                        h1.SetBinError(bidx_l, 0.3*h1.GetBinContent(bidx_l))
                h1.Write()
            file.Close()

CPU times: user 2min 6s, sys: 35.6 s, total: 2min 42s
Wall time: 3h 11min 54s


Warning in <TClass::Init>: no dictionary for class edm::Hash<1> is available
Warning in <TClass::Init>: no dictionary for class edm::ProcessHistory is available
Warning in <TClass::Init>: no dictionary for class edm::ProcessConfiguration is available
Warning in <TClass::Init>: no dictionary for class edm::ParameterSetBlob is available
Warning in <TClass::Init>: no dictionary for class pair<edm::Hash<1>,edm::ParameterSetBlob> is available


# Create and save stackplots

In [19]:
#%load_ext autoreload
#%autoreload 2
#h

In [20]:
if "stack" not in os.listdir("./" + results_dir):
    os.mkdir("./" + results_dir + "/stack")
for region in ["SR", "CR_non_prompt", #"CR_QCD", 
               "CR_ttbar", "CR_wrong_sign", "CR_Zjets"]:
    for v in variables[region]:
        if region != "SR":
            stackplot(region, v, "etau", "./" + results_dir + "/stack", h)
            stackplot(region, v, "mutau", "./" + results_dir +"/stack", h) 
        else:
            stackplot(region, v, "etau", "./" + results_dir + "/stack", h, blinded = True)
            stackplot(region, v, "mutau", "./" + results_dir + "/stack", h, blinded = True) 

SR
SR
SR
SR
SR
SR
SR
SR
SR
SR
SR
SR
CR_non_prompt
CR_non_prompt
CR_non_prompt
CR_non_prompt
CR_non_prompt
CR_non_prompt
CR_non_prompt
CR_non_prompt
CR_non_prompt
CR_non_prompt
CR_non_prompt
CR_non_prompt
CR_ttbar
CR_ttbar
CR_ttbar
CR_ttbar
CR_ttbar
CR_ttbar
CR_ttbar
CR_ttbar
CR_ttbar
CR_ttbar
CR_ttbar
CR_ttbar
CR_wrong_sign
CR_wrong_sign
CR_wrong_sign
CR_wrong_sign
CR_wrong_sign
CR_wrong_sign
CR_wrong_sign
CR_wrong_sign
CR_wrong_sign
CR_wrong_sign
CR_wrong_sign
CR_wrong_sign
CR_Zjets
CR_Zjets
CR_Zjets
CR_Zjets
CR_Zjets
CR_Zjets
CR_Zjets
CR_Zjets
CR_Zjets
CR_Zjets
CR_Zjets
CR_Zjets


Info in <TCanvas::Print>: png file ./resultsRR2017_af-test_fix/stack/SR/etau/Leadingjet_pt.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: png file ./resultsRR2017_af-test_fix/stack/SR/mutau/Leadingjet_pt.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: png file ./resultsRR2017_af-test_fix/stack/SR/etau/SubLeadingjet_pt.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: png file ./resultsRR2017_af-test_fix/stack/SR/mutau/SubLeadingjet_pt.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: png file ./resultsRR2017_af-test_fix/stack/SR/etau/Lepton_pt.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: png file ./resultsRR2017_af-test_fix/sta